In [1]:
import os

In [57]:
%pwd

'c:\\Users\\heman\\Downloads\\kidney project\\kidney-disease-classification-Mlflow-DVC'

In [3]:
os.chdir("../")

In [58]:
%pwd

'c:\\Users\\heman\\Downloads\\kidney project\\kidney-disease-classification-Mlflow-DVC'

In [ ]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Hemu7/Kidney-disease-classification-Mlflow-DVC123.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Hemu7"
os.environ["MLFLOW_TRACKING_PASSWORD"]="Hemanth@3145749941"

In [75]:
import tensorflow as tf


In [76]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [77]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [78]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [79]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney",
            mlflow_uri="https://dagshub.com/Hemu7/Kidney-disease-classification-Mlflow-DVC123.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [80]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse


import os
import mlflow
import mlflow.keras
from urllib.parse import urlparse


In [81]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)
        
        
    def log_into_mlflow(self):
        import os
        import mlflow
        import mlflow.keras
        from urllib.parse import urlparse

        # make sure credentials are loaded (optional if set globally)
        os.environ["MLFLOW_TRACKING_USERNAME"] = "Hemu7"
        os.environ["MLFLOW_TRACKING_PASSWORD"] = "Hemanth@3145749941"

        # Set tracking URI (not registry URI)
        mlflow.set_tracking_uri(self.config.mlflow_uri)

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )

            # log model
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(
                    self.model,
                    "model",
                    registered_model_name="VGG16Model"
                )
            else:
                mlflow.keras.log_model(self.model, "model")

        
    
    

    
    

In [82]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-09-12 13:42:56,638: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-12 13:42:56,653: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-12 13:42:56,654: INFO: common: created directory at: artifacts]
Found 2207 images belonging to 2 classes.
138/138 [==============================] - 126s 909ms/step - loss: 5.6904 - accuracy: 0.4300
[2025-09-12 13:45:02,921: INFO: common: json file saved at: scores.json]


2025/09/12 13:45:03 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-09-12 13:45:05,514: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\heman\AppData\Local\Temp\tmpx83_80he\model\data\model\assets
[2025-09-12 13:45:05,959: INFO: builder_impl: Assets written to: C:\Users\heman\AppData\Local\Temp\tmpx83_80he\model\data\model\assets]


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/09/12 13:45:35 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 4
Created version '4' of model 'VGG16Model'.
